In [55]:
import pandas as pd
import geopandas as gpd
from pyproj import CRS
from shapely.geometry import Polygon
from shapely.geometry import Point
import rasterio as rio
from rasterio import features
from rasterio.windows import Window
import matplotlib.pyplot as plt
import numpy as np
import pprint as pp
import itertools as it
from tqdm import tqdm
import time

from leaf.deforestation import (
    area,
    to_series,
    closest_index,
)


In [57]:
geo = gpd.read_file('data/Hansen_GFC-2022-v1.10_lossyear_20S_060W.gpkg')
asset = pd.read_csv('data/combined_asset_data.csv')

In [59]:
brazil = asset[asset['country'] == 'Brazil']
#blip = brazil[(brazil['longitude'] > -60) & (brazil['longitude'] < -58.8) & (brazil['latitude'] > -21) & (brazil['latitude'] < -20)]
brazil.head()

,uid,asset_name,country,start_year,latitude,longitude,parent_name,owner_name,operator_name,sector,data_source
153,GEM_167,Afonso Bezerra wind farm,Brazil,2022.0,-5.3246,-36.5062,NaN,QAir Brasil,QAir Brasil,wind power/onshore,GEM
154,GEM_168,Afonso Bezerra wind farm,Brazil,2022.0,-5.3612,-36.4723,NaN,QAir Brasil,QAir Brasil,wind power/onshore,GEM
155,GEM_169,Afonso Bezerra wind farm,Brazil,2022.0,-5.4091,-36.4925,NaN,QAir Brasil,QAir Brasil,wind power/onshore,GEM
156,GEM_170,Afonso Bezerra wind farm,Brazil,2022.0,-5.3504,-36.4693,NaN,QAir Brasil,QAir Brasil,wind power/onshore,GEM
157,GEM_171,Afonso Bezerra wind farm,Brazil,2022.0,-5.3151,-36.4892,NaN,QAir Brasil,QAir Brasil,wind power/onshore,GEM


In [ ]:

from typing import Tuple, Optional

def closest_(gdf: gpd.GeoDataFrame, lat: float, long: float, year: int, verbose: bool = False) -> Tuple[float, int]:

    # TODO: geopandas.sindex.SpatialIndex.nearest
    distances = gdf[gdf['lossyear']==year].distance(Point(lat, long))
    index = distances.idxmin()
    return (distances.iloc[index], index)

proj_3857 = geo.to_crs(epsg=3347) # lambert projection
#foo = closest_(proj_3857, -22, -55, 2020, False)
#print(foo)
#geo.iloc[foo[1]]
temp = brazil.apply(lambda x: closest_(proj_3857, x.latitude, x.longitude, 2020, False), axis=1)
geo.iloc[temp[1]]

In [64]:
geometry=[Point(xy) for xy in zip(brazil["longitude"], brazil["latitude"])]
geodata=gpd.GeoDataFrame(brazil,crs=geo.crs, geometry=geometry)
base = geo[geo.lossyear==2020].plot(column='group', legend=True)
geodata[geodata.start_year==2020].plot(ax=base, marker='o', color='red', markersize=5);


In [63]:
geodata.columns

Index(['uid', 'asset_name', 'country', 'start_year', 'latitude', 'longitude',
       'parent_name', 'owner_name', 'operator_name', 'sector', 'data_source',
       'geometry'],
      dtype='object')